In [1]:
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.coreference_classifier import CoreferenceClassifierModelBuilder
from functools import reduce
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
markables = get_markable_dataframe("data/training/markables.csv", word_vector, idx_by_word)

markables.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1,[1],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,[],"[2, 3, 4, 5, 6, 7, 8, 9, 10, 2]","[0.0, 1.0]"
1,2,"[4, 5]",0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,"[1, 2, 3]","[6, 7, 8, 9, 10, 2, 11, 12, 13, 14]","[0.0, 1.0]"
2,3,"[6, 7, 8]",0,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]",1,0,"[1, 2, 3, 4, 5]","[9, 10, 2, 11, 12, 13, 14, 15, 16, 17]","[1.0, 0.0]"
3,4,[10],0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,"[1, 2, 3, 4, 5, 6, 7, 8, 9]","[2, 11, 12, 13, 14, 15, 16, 17, 18, 19]","[0.0, 1.0]"
4,5,"[12, 13]",0,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",0,0,"[3, 4, 5, 6, 7, 8, 9, 10, 2, 11]","[14, 15, 16, 17, 18, 19, 4, 5, 20, 21]","[0.0, 1.0]"


In [4]:
# pairs_budi = pd.read_csv("data/training/mention_pairs_budi.csv")
pairs_soon = pd.read_csv("data/training/mention_pairs_soon.csv")
pairs_gilang = pd.read_csv("data/training/mention_pairs_gilang.csv")

pairs_soon.head()

,m1_id,m2_id,is_exact_match,is_words_match,is_substring,is_abbreviation,is_appositive,is_nearest_candidate,sentence_distance,word_distance,markable_distance,is_coreference
0,3,11,0,0,0,0,0,0,2,31,8,1
1,4,11,0,0,0,0,0,0,2,29,7,0
2,5,11,0,0,0,0,0,0,2,25,6,0
3,6,11,0,0,0,0,0,0,2,16,5,0
4,7,11,0,0,0,0,0,0,1,14,4,0


In [5]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

def get_data(markable_ids):
    indices = reduce(lambda a, b: a + [b], map(lambda a: markables.index[markables['id'] == a].tolist()[0], markable_ids), [])
    data = markables.loc[indices]
    
    data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
    data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
    data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
    data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

    data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
    is_singleton = np.vstack(data.is_singleton)
    
    return data_text, data_previous_words, data_next_words, data_syntactic, is_singleton

def get_pair_data(markable_ids_1, markable_ids_2):
    text_1, prev_1, next_1, syntactic_1, is_singleton_1 = get_data(markable_ids_1)
    text_2, prev_2, next_2, syntactic_2, is_singleton_2 = get_data(markable_ids_2)
    
    return text_1, text_2, prev_1, prev_2, next_1, next_2, syntactic_1, syntactic_2, is_singleton_1, is_singleton_2

def get_relation_data(mention_pairs):
    return mention_pairs[['is_exact_match', 'is_words_match', 'is_substring', 'is_abbreviation', 'is_appositive', 'is_nearest_candidate', 'sentence_distance', 'word_distance', 'markable_distance']]

# Soon-Generated

In [6]:
text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, is_singleton_11, is_singleton_21 = get_pair_data(pairs_soon.m1_id, pairs_soon.m2_id)
relation_1 = get_relation_data(pairs_soon)
label_soon = np.vstack(to_categorical(pairs_soon.is_coreference, num_classes=2))

## Build Model

### Word

In [7]:
words_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix
)

In [8]:
words_model_1 = words_model_builder.create_model()

W0314 11:36:07.803669 140080703129408 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:111: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0314 11:36:09.137536 140080703129408 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:135: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0314 11:36:09.194277 140080703129408 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1253: calling VarianceScaling.__init__ (from tensorflow.python.op

In [9]:
words_model_1.fit([text_11, text_21], label_soon, epochs=5)
words_model_1.save('models/coreference_classifiers/words_soon_5.model')

Epoch 1/5
5548/5548==============================] - 3s 524us/sample - loss: 0.3153 - acc: 0.8807
Epoch 2/5
5548/5548==============================] - 2s 324us/sample - loss: 0.2306 - acc: 0.8884
Epoch 3/5
5548/5548==============================] - 2s 321us/sample - loss: 0.2052 - acc: 0.8888
Epoch 4/5
5548/5548==============================] - 2s 323us/sample - loss: 0.1902 - acc: 0.8980
Epoch 5/5
5548/5548==============================] - 2s 321us/sample - loss: 0.1773 - acc: 0.9106


In [10]:
words_model_1.fit([text_11, text_21], label_soon, epochs=5)
words_model_1.save('models/coreference_classifiers/words_soon_10.model')

Epoch 1/5
5548/5548==============================] - 2s 324us/sample - loss: 0.1692 - acc: 0.9230
Epoch 2/5
5548/5548==============================] - 2s 327us/sample - loss: 0.1512 - acc: 0.9274
Epoch 3/5
5548/5548==============================] - 2s 322us/sample - loss: 0.1464 - acc: 0.9268
Epoch 4/5
5548/5548==============================] - 2s 323us/sample - loss: 0.1381 - acc: 0.9328
Epoch 5/5
5548/5548==============================] - 2s 323us/sample - loss: 0.1257 - acc: 0.9376


In [11]:
words_model_1.fit([text_11, text_21], label_soon, epochs=10)
words_model_1.save('models/coreference_classifiers/words_soon_20.model')

Epoch 1/10
5548/5548==============================] - 2s 323us/sample - loss: 0.1210 - acc: 0.9429
Epoch 2/10
5548/5548==============================] - 2s 327us/sample - loss: 0.1132 - acc: 0.9411
Epoch 3/10
5548/5548==============================] - 2s 322us/sample - loss: 0.1217 - acc: 0.9407
Epoch 4/10
5548/5548==============================] - 2s 322us/sample - loss: 0.1057 - acc: 0.9508
Epoch 5/10
5548/5548==============================] - 2s 321us/sample - loss: 0.1051 - acc: 0.9506
Epoch 6/10
5548/5548==============================] - 2s 319us/sample - loss: 0.1005 - acc: 0.9513
Epoch 7/10
5548/5548==============================] - 2s 323us/sample - loss: 0.1010 - acc: 0.9508
Epoch 8/10
5548/5548==============================] - 2s 321us/sample - loss: 0.1214 - acc: 0.9557
Epoch 9/10
5548/5548==============================] - 2s 321us/sample - loss: 0.0977 - acc: 0.9530
Epoch 10/10
5548/5548==============================] - 2s 322us/sample - loss: 0.0861 - acc: 0.9575


### Context

In [12]:
context_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix
)

In [13]:
context_model_1 = context_model_builder.create_model()

In [14]:
context_model_1.fit([prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
context_model_1.save('models/coreference_classifiers/context_soon_5.model')

Epoch 1/5
5548/5548==============================] - 3s 582us/sample - loss: 0.4483 - acc: 0.8300
Epoch 2/5
5548/5548==============================] - 3s 494us/sample - loss: 0.3451 - acc: 0.8872
Epoch 3/5
5548/5548==============================] - 3s 497us/sample - loss: 0.3065 - acc: 0.8873
Epoch 4/5
5548/5548==============================] - 3s 489us/sample - loss: 0.2707 - acc: 0.8875
Epoch 5/5
5548/5548==============================] - 3s 497us/sample - loss: 0.2401 - acc: 0.8872


In [15]:
context_model_1.fit([prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
context_model_1.save('models/coreference_classifiers/context_soon_10.model')

Epoch 1/5
5548/5548==============================] - 3s 490us/sample - loss: 0.2168 - acc: 0.8873
Epoch 2/5
5548/5548==============================] - 3s 490us/sample - loss: 0.1974 - acc: 0.8992
Epoch 3/5
5548/5548==============================] - 3s 492us/sample - loss: 0.1914 - acc: 0.9115
Epoch 4/5
5548/5548==============================] - 3s 492us/sample - loss: 0.1810 - acc: 0.9160
Epoch 5/5
5548/5548==============================] - 3s 494us/sample - loss: 0.1698 - acc: 0.9211


In [16]:
context_model_1.fit([prev_11, prev_21, next_11, next_21], label_soon, epochs=10)
context_model_1.save('models/coreference_classifiers/context_soon_20.model')

Epoch 1/10
5548/5548==============================] - 3s 489us/sample - loss: 0.1570 - acc: 0.9257
Epoch 2/10
5548/5548==============================] - 3s 501us/sample - loss: 0.1555 - acc: 0.9297
Epoch 3/10
5548/5548==============================] - 3s 490us/sample - loss: 0.1532 - acc: 0.9328
Epoch 4/10
5548/5548==============================] - 3s 493us/sample - loss: 0.1491 - acc: 0.9322
Epoch 5/10
5548/5548==============================] - 3s 491us/sample - loss: 0.1381 - acc: 0.9353
Epoch 6/10
5548/5548==============================] - 3s 492us/sample - loss: 0.1356 - acc: 0.9380
Epoch 7/10
5548/5548==============================] - 3s 488us/sample - loss: 0.1335 - acc: 0.9445
Epoch 8/10
5548/5548==============================] - 3s 501us/sample - loss: 0.1296 - acc: 0.9405
Epoch 9/10
5548/5548==============================] - 3s 491us/sample - loss: 0.1235 - acc: 0.9486
Epoch 10/10
5548/5548==============================] - 3s 491us/sample - loss: 0.1234 - acc: 0.9479


### Syntactic

In [17]:
syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [18]:
syntactic_model_1 = syntactic_model_builder.create_model()

In [ ]:
syntactic_model_1.fit([syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
syntactic_model_1.save('models/coreference_classifiers/syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 2s 307us/sample - loss: 0.4452 - acc: 0.8482
Epoch 2/5
5548/5548==============================] - 1s 257us/sample - loss: 0.2707 - acc: 0.8873
Epoch 3/5
5548/5548==============================] - 1s 254us/sample - loss: 0.2482 - acc: 0.8857
Epoch 4/5
5548/5548==============================] - 1s 255us/sample - loss: 0.2329 - acc: 0.8873
Epoch 5/5
5548/5548==============================] - 1s 249us/sample - loss: 0.2235 - acc: 0.8875


In [ ]:
syntactic_model_1.fit([syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
syntactic_model_1.save('models/coreference_classifiers/syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 1s 247us/sample - loss: 0.2113 - acc: 0.8891
Epoch 2/5
5548/5548==============================] - 1s 252us/sample - loss: 0.1943 - acc: 0.9030
Epoch 3/5
5548/5548==============================] - 1s 249us/sample - loss: 0.1854 - acc: 0.9185
Epoch 4/5
5548/5548==============================] - 1s 253us/sample - loss: 0.1817 - acc: 0.9205
Epoch 5/5
5548/5548==============================] - 1s 247us/sample - loss: 0.1733 - acc: 0.9272


In [ ]:
syntactic_model_1.fit([syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
syntactic_model_1.save('models/coreference_classifiers/syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 1s 248us/sample - loss: 0.1675 - acc: 0.9279
Epoch 2/10
5548/5548==============================] - 1s 253us/sample - loss: 0.1673 - acc: 0.9302
Epoch 3/10
5548/5548==============================] - 1s 254us/sample - loss: 0.1609 - acc: 0.9310
Epoch 4/10
5548/5548==============================] - 1s 251us/sample - loss: 0.1583 - acc: 0.9346
Epoch 5/10
5548/5548==============================] - 1s 252us/sample - loss: 0.1563 - acc: 0.9297
Epoch 6/10
5548/5548==============================] - 1s 259us/sample - loss: 0.1606 - acc: 0.9313
Epoch 7/10
5548/5548==============================] - 1s 257us/sample - loss: 0.1479 - acc: 0.9371
Epoch 8/10
5548/5548==============================] - 1s 253us/sample - loss: 0.1534 - acc: 0.9328
Epoch 9/10
5548/5548==============================] - 1s 252us/sample - loss: 0.1486 - acc: 0.9319
Epoch 10/10
5548/5548==============================] - 1s 251us/sample - loss: 0.1520 - acc: 0.9329


### Words + Context

In [ ]:
words_context_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=False,
    use_relation_feature=False,
    embedding_matrix=embedding_matrix,
)

In [ ]:
words_context_model_1 = words_context_model_builder.create_model()

In [ ]:
words_context_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
words_context_model_1.save('models/coreference_classifiers/words_context_soon_5.model')

Epoch 1/5
5548/5548==============================] - 5s 850us/sample - loss: 0.3456 - acc: 0.8776
Epoch 2/5
5548/5548==============================] - 4s 704us/sample - loss: 0.2450 - acc: 0.8967
Epoch 3/5
5548/5548==============================] - 4s 708us/sample - loss: 0.1989 - acc: 0.9151
Epoch 4/5
5548/5548==============================] - 4s 711us/sample - loss: 0.1706 - acc: 0.9214
Epoch 5/5
5548/5548==============================] - 4s 706us/sample - loss: 0.1489 - acc: 0.9351


In [ ]:
words_context_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21], label_soon, epochs=5)
words_context_model_1.save('models/coreference_classifiers/words_context_soon_10.model')

Epoch 1/5
5548/5548==============================] - 4s 692us/sample - loss: 0.1342 - acc: 0.9394
Epoch 2/5
5548/5548==============================] - 4s 704us/sample - loss: 0.1198 - acc: 0.9468
Epoch 3/5
5548/5548==============================] - 4s 716us/sample - loss: 0.1075 - acc: 0.9539
Epoch 4/5
5548/5548==============================] - 4s 710us/sample - loss: 0.0989 - acc: 0.9621
Epoch 5/5
5548/5548==============================] - 4s 708us/sample - loss: 0.0816 - acc: 0.9688


In [ ]:
words_context_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21], label_soon, epochs=10)
words_context_model_1.save('models/coreference_classifiers/words_context_soon_20.model')

Epoch 1/10
5548/5548==============================] - 4s 692us/sample - loss: 0.0694 - acc: 0.9757
Epoch 2/10
5548/5548==============================] - 4s 701us/sample - loss: 0.0749 - acc: 0.9744
Epoch 3/10
5548/5548==============================] - 4s 709us/sample - loss: 0.0599 - acc: 0.9804
Epoch 4/10
5548/5548==============================] - 4s 703us/sample - loss: 0.0574 - acc: 0.9804
Epoch 5/10
5548/5548==============================] - 4s 702us/sample - loss: 0.0622 - acc: 0.9818
Epoch 6/10
5548/5548==============================] - 4s 709us/sample - loss: 0.0486 - acc: 0.9834
Epoch 7/10
5548/5548==============================] - 4s 710us/sample - loss: 0.0440 - acc: 0.9834
Epoch 8/10
5548/5548==============================] - 4s 703us/sample - loss: 0.0422 - acc: 0.9863
Epoch 9/10
5548/5548==============================] - 4s 712us/sample - loss: 0.0450 - acc: 0.9852
Epoch 10/10
5548/5548==============================] - 4s 707us/sample - loss: 0.0381 - acc: 0.9870


### Words + Syntactic

In [ ]:
words_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=False, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [ ]:
words_syntactic_model_1 = words_syntactic_model_builder.create_model()

In [ ]:
words_syntactic_model_1.fit([text_11, text_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_syntactic_model_1.save('models/coreference_classifiers/words_syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 3s 565us/sample - loss: 0.4662 - acc: 0.7868
Epoch 2/5
5548/5548==============================] - 3s 452us/sample - loss: 0.2288 - acc: 0.8879
Epoch 3/5
5548/5548==============================] - 3s 457us/sample - loss: 0.1865 - acc: 0.9021
Epoch 4/5
5548/5548==============================] - 3s 456us/sample - loss: 0.1572 - acc: 0.9257
Epoch 5/5
5548/5548==============================] - 3s 456us/sample - loss: 0.1357 - acc: 0.9409


In [ ]:
words_syntactic_model_1.fit([text_11, text_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_syntactic_model_1.save('models/coreference_classifiers/words_syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 2s 447us/sample - loss: 0.1156 - acc: 0.9531
Epoch 2/5
5548/5548==============================] - 3s 455us/sample - loss: 0.1039 - acc: 0.9575
Epoch 3/5
5548/5548==============================] - 3s 452us/sample - loss: 0.1002 - acc: 0.9596
Epoch 4/5
5548/5548==============================] - 3s 459us/sample - loss: 0.0841 - acc: 0.9656
Epoch 5/5
5548/5548==============================] - 3s 456us/sample - loss: 0.0770 - acc: 0.9667


In [ ]:
words_syntactic_model_1.fit([text_11, text_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
words_syntactic_model_1.save('models/coreference_classifiers/words_syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 3s 452us/sample - loss: 0.0734 - acc: 0.9694
Epoch 2/10
5548/5548==============================] - 3s 456us/sample - loss: 0.0777 - acc: 0.9695
Epoch 3/10
5548/5548==============================] - 3s 456us/sample - loss: 0.0678 - acc: 0.9735
Epoch 4/10
5548/5548==============================] - 3s 456us/sample - loss: 0.0753 - acc: 0.9699
Epoch 5/10
5548/5548==============================] - 3s 458us/sample - loss: 0.0677 - acc: 0.9730
Epoch 6/10
5548/5548==============================] - 3s 454us/sample - loss: 0.0577 - acc: 0.9757
Epoch 7/10
5548/5548==============================] - 3s 457us/sample - loss: 0.0585 - acc: 0.9740
Epoch 8/10
5548/5548==============================] - 3s 462us/sample - loss: 0.0571 - acc: 0.9762
Epoch 9/10
5548/5548==============================] - 3s 456us/sample - loss: 0.0547 - acc: 0.9766
Epoch 10/10
5548/5548==============================] - 3s 463us/sample - loss: 0.0507 - acc: 0.9771


### Context + Syntactic

In [ ]:
context_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=False, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [ ]:
context_syntactic_model_1 = context_syntactic_model_builder.create_model()

In [ ]:
context_syntactic_model_1.fit([prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
context_syntactic_model_1.save('models/coreference_classifiers/context_syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 4s 784us/sample - loss: 0.4148 - acc: 0.8774
Epoch 2/5
5548/5548==============================] - 4s 637us/sample - loss: 0.2680 - acc: 0.8868
Epoch 3/5
5548/5548==============================] - 3s 630us/sample - loss: 0.2470 - acc: 0.8879
Epoch 4/5
5548/5548==============================] - 3s 630us/sample - loss: 0.2315 - acc: 0.8875
Epoch 5/5
5548/5548==============================] - 4s 633us/sample - loss: 0.2178 - acc: 0.8884


In [ ]:
context_syntactic_model_1.fit([prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
context_syntactic_model_1.save('models/coreference_classifiers/context_syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 3s 619us/sample - loss: 0.2077 - acc: 0.8951
Epoch 2/5
5548/5548==============================] - 4s 633us/sample - loss: 0.1980 - acc: 0.9043
Epoch 3/5
5548/5548==============================] - 3s 630us/sample - loss: 0.1815 - acc: 0.9133
Epoch 4/5
5548/5548==============================] - 3s 628us/sample - loss: 0.1755 - acc: 0.9252
Epoch 5/5
5548/5548==============================] - 4s 634us/sample - loss: 0.1630 - acc: 0.9328


In [ ]:
context_syntactic_model_1.fit([prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
context_syntactic_model_1.save('models/coreference_classifiers/context_syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 3s 614us/sample - loss: 0.1458 - acc: 0.9380
Epoch 2/10
5548/5548==============================] - 3s 627us/sample - loss: 0.1328 - acc: 0.9465
Epoch 3/10
5548/5548==============================] - 3s 630us/sample - loss: 0.1163 - acc: 0.9528
Epoch 4/10
5548/5548==============================] - 4s 636us/sample - loss: 0.1042 - acc: 0.9575
Epoch 5/10
5548/5548==============================] - 3s 631us/sample - loss: 0.1003 - acc: 0.9603
Epoch 6/10
5548/5548==============================] - 3s 627us/sample - loss: 0.0990 - acc: 0.9602
Epoch 7/10
5548/5548==============================] - 3s 621us/sample - loss: 0.0832 - acc: 0.9679
Epoch 8/10
5548/5548==============================] - 3s 630us/sample - loss: 0.0779 - acc: 0.9699
Epoch 9/10
5548/5548==============================] - 3s 630us/sample - loss: 0.0780 - acc: 0.9706
Epoch 10/10
5548/5548==============================] - 4s 636us/sample - loss: 0.0687 - acc: 0.9744


### Words + Context + Syntactic

In [ ]:
words_context_syntactic_model_builder = CoreferenceClassifierModelBuilder(
    use_words_feature=True, 
    use_context_feature=True, 
    use_syntactic_feature=True,
    use_relation_feature=True,
    embedding_matrix=embedding_matrix,
    syntactic_features_num=syntactic_11.shape[1],
    relation_features_num=relation_1.shape[1]
)

In [ ]:
words_context_syntactic_model_1 = words_context_syntactic_model_builder.create_model()

In [ ]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_5.model')

Epoch 1/5
5548/5548==============================] - 5s 989us/sample - loss: 0.5023 - acc: 0.8598
Epoch 2/5
5548/5548==============================] - 4s 795us/sample - loss: 0.2335 - acc: 0.8917
Epoch 3/5
5548/5548==============================] - 4s 798us/sample - loss: 0.2010 - acc: 0.9009
Epoch 4/5
5548/5548==============================] - 4s 804us/sample - loss: 0.1703 - acc: 0.9182
Epoch 5/5
5548/5548==============================] - 4s 799us/sample - loss: 0.1536 - acc: 0.9304


In [ ]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=5)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_10.model')

Epoch 1/5
5548/5548==============================] - 4s 775us/sample - loss: 0.1276 - acc: 0.9463
Epoch 2/5
5548/5548==============================] - 4s 800us/sample - loss: 0.1111 - acc: 0.9555
Epoch 3/5
5548/5548==============================] - 4s 794us/sample - loss: 0.0995 - acc: 0.9627
Epoch 4/5
5548/5548==============================] - 4s 793us/sample - loss: 0.0757 - acc: 0.9726
Epoch 5/5
5548/5548==============================] - 4s 791us/sample - loss: 0.0707 - acc: 0.9731


In [ ]:
words_context_syntactic_model_1.fit([text_11, text_21, prev_11, prev_21, next_11, next_21, syntactic_11, syntactic_21, relation_1], label_soon, epochs=10)
words_context_syntactic_model_1.save('models/coreference_classifiers/words_context_syntactic_soon_20.model')

Epoch 1/10
5548/5548==============================] - 4s 792us/sample - loss: 0.0601 - acc: 0.9789
Epoch 2/10
5548/5548==============================] - 4s 810us/sample - loss: 0.0534 - acc: 0.9816
Epoch 3/10
5548/5548==============================] - 4s 808us/sample - loss: 0.0556 - acc: 0.9793
Epoch 4/10
5548/5548==============================] - 4s 793us/sample - loss: 0.0466 - acc: 0.9825
Epoch 5/10
5548/5548==============================] - 4s 804us/sample - loss: 0.0479 - acc: 0.9827
Epoch 6/10
5548/5548==============================] - 4s 794us/sample - loss: 0.0297 - acc: 0.9897
Epoch 7/10
5548/5548==============================] - 4s 810us/sample - loss: 0.0287 - acc: 0.9899
Epoch 8/10
5548/5548==============================] - 4s 792us/sample - loss: 0.0376 - acc: 0.9894
Epoch 9/10
5548/5548==============================] - 4s 801us/sample - loss: 0.0232 - acc: 0.9924
Epoch 10/10
5548/5548==============================] - 4s 804us/sample - loss: 0.0158 - acc: 0.9953


# Gilang-Generated

In [ ]:
text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, is_singleton_12, is_singleton_22 = get_pair_data(pairs_gilang.m1_id, pairs_gilang.m2_id)
relation_2 = get_relation_data(pairs_gilang)
label_gilang = np.vstack(to_categorical(pairs_gilang.is_coreference, num_classes=2))

## Build Model

### Words

In [ ]:
words_model_2 = words_model_builder.create_model()

In [ ]:
words_model_2.fit([text_12, text_22], label_gilang, epochs=5)
words_model_2.save('models/coreference_classifiers/words_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 14s 376us/sample - loss: 0.0731 - acc: 0.9773
Epoch 2/5
36503/36503==============================] - 13s 359us/sample - loss: 0.0445 - acc: 0.9829
Epoch 3/5
36503/36503==============================] - 13s 362us/sample - loss: 0.0405 - acc: 0.9829
Epoch 4/5
36503/36503==============================] - 13s 359us/sample - loss: 0.0366 - acc: 0.9830
Epoch 5/5
36503/36503==============================] - 13s 360us/sample - loss: 0.0366 - acc: 0.9830


In [ ]:
words_model_2.fit([text_12, text_22], label_gilang, epochs=5)
words_model_2.save('models/coreference_classifiers/words_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 13s 358us/sample - loss: 0.0339 - acc: 0.9830
Epoch 2/5
36503/36503==============================] - 13s 361us/sample - loss: 0.0349 - acc: 0.9830
Epoch 3/5
36503/36503==============================] - 13s 359us/sample - loss: 0.0324 - acc: 0.9830
Epoch 4/5
36503/36503==============================] - 13s 360us/sample - loss: 0.0342 - acc: 0.9830
Epoch 5/5
36503/36503==============================] - 13s 358us/sample - loss: 0.0307 - acc: 0.9830


In [ ]:
words_model_2.fit([text_12, text_22], label_gilang, epochs=10)
words_model_2.save('models/coreference_classifiers/words_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 13s 359us/sample - loss: 0.0339 - acc: 0.9838
Epoch 2/10
36503/36503==============================] - 13s 358us/sample - loss: 0.0297 - acc: 0.9841
Epoch 3/10
36503/36503==============================] - 13s 359us/sample - loss: 0.0281 - acc: 0.9850
Epoch 4/10
36503/36503==============================] - 13s 360us/sample - loss: 0.0284 - acc: 0.9852
Epoch 5/10
36503/36503==============================] - 13s 359us/sample - loss: 0.0270 - acc: 0.9856
Epoch 6/10
36503/36503==============================] - 13s 359us/sample - loss: 0.0276 - acc: 0.9854
Epoch 7/10
36503/36503==============================] - 13s 358us/sample - loss: 0.0270 - acc: 0.9854
Epoch 8/10
36503/36503==============================] - 13s 356us/sample - loss: 0.0265 - acc: 0.9856
Epoch 9/10
36503/36503==============================] - 13s 359us/sample - loss: 0.0264 - acc: 0.9854
Epoch 10/10
36503/36503==============================] - 13s 358us/sample - loss: 

### Context

In [ ]:
context_model_2 = context_model_builder.create_model()

In [ ]:
context_model_2.fit([prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
context_model_2.save('models/coreference_classifiers/context_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 21s 566us/sample - loss: 0.0998 - acc: 0.9827
Epoch 2/5
36503/36503==============================] - 20s 542us/sample - loss: 0.0735 - acc: 0.9829
Epoch 3/5
36503/36503==============================] - 20s 539us/sample - loss: 0.0604 - acc: 0.9830
Epoch 4/5
36503/36503==============================] - 20s 540us/sample - loss: 0.0545 - acc: 0.9827
Epoch 5/5
36503/36503==============================] - 20s 545us/sample - loss: 0.0480 - acc: 0.9829


In [ ]:
context_model_2.fit([prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
context_model_2.save('models/coreference_classifiers/context_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 20s 541us/sample - loss: 0.0430 - acc: 0.9830
Epoch 2/5
36503/36503==============================] - 20s 544us/sample - loss: 0.0417 - acc: 0.9848
Epoch 3/5
36503/36503==============================] - 20s 542us/sample - loss: 0.0372 - acc: 0.9859
Epoch 4/5
36503/36503==============================] - 20s 543us/sample - loss: 0.0341 - acc: 0.9869
Epoch 5/5
36503/36503==============================] - 20s 539us/sample - loss: 0.0326 - acc: 0.9887


In [ ]:
context_model_2.fit([prev_12, prev_22, next_12, next_22], label_gilang, epochs=10)
context_model_2.save('models/coreference_classifiers/context_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 20s 543us/sample - loss: 0.0303 - acc: 0.9886
Epoch 2/10
36503/36503==============================] - 20s 542us/sample - loss: 0.0300 - acc: 0.9889
Epoch 3/10
36503/36503==============================] - 20s 542us/sample - loss: 0.0292 - acc: 0.9896
Epoch 4/10
36503/36503==============================] - 20s 540us/sample - loss: 0.0258 - acc: 0.9908
Epoch 5/10
36503/36503==============================] - 20s 544us/sample - loss: 0.0259 - acc: 0.9905
Epoch 6/10
36503/36503==============================] - 20s 544us/sample - loss: 0.0249 - acc: 0.9907
Epoch 7/10
36503/36503==============================] - 20s 540us/sample - loss: 0.0255 - acc: 0.9908
Epoch 8/10
36503/36503==============================] - 20s 544us/sample - loss: 0.0251 - acc: 0.9909
Epoch 9/10
36503/36503==============================] - 20s 541us/sample - loss: 0.0225 - acc: 0.9919
Epoch 10/10
36503/36503==============================] - 20s 540us/sample - loss: 

### Syntactic

In [ ]:
syntactic_model_2 = syntactic_model_builder.create_model()

In [ ]:
syntactic_model_2.fit([syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
syntactic_model_2.save('models/coreference_classifiers/syntactic_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 11s 300us/sample - loss: 0.2859 - acc: 0.9575
Epoch 2/5
36503/36503==============================] - 10s 284us/sample - loss: 0.0517 - acc: 0.9827
Epoch 3/5
36503/36503==============================] - 10s 283us/sample - loss: 0.0488 - acc: 0.9825
Epoch 4/5
36503/36503==============================] - 10s 286us/sample - loss: 0.0464 - acc: 0.9824
Epoch 5/5
36503/36503==============================] - 10s 285us/sample - loss: 0.0424 - acc: 0.9827


In [ ]:
syntactic_model_2.fit([syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
syntactic_model_2.save('models/coreference_classifiers/syntactic_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 10s 285us/sample - loss: 0.0424 - acc: 0.9834
Epoch 2/5
36503/36503==============================] - 10s 285us/sample - loss: 0.0392 - acc: 0.9843
Epoch 3/5
36503/36503==============================] - 10s 285us/sample - loss: 0.0400 - acc: 0.9838
Epoch 4/5
36503/36503==============================] - 10s 282us/sample - loss: 0.0395 - acc: 0.9840
Epoch 5/5
36503/36503==============================] - 10s 284us/sample - loss: 0.0386 - acc: 0.9843


In [ ]:
syntactic_model_2.fit([syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
syntactic_model_2.save('models/coreference_classifiers/syntactic_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 10s 283us/sample - loss: 0.0393 - acc: 0.9844
Epoch 2/10
36503/36503==============================] - 10s 284us/sample - loss: 0.0378 - acc: 0.9838
Epoch 3/10
36503/36503==============================] - 10s 283us/sample - loss: 0.0380 - acc: 0.9843
Epoch 4/10
36503/36503==============================] - 10s 286us/sample - loss: 0.0376 - acc: 0.9847
Epoch 5/10
36503/36503==============================] - 10s 285us/sample - loss: 0.0371 - acc: 0.9850
Epoch 6/10
36503/36503==============================] - 10s 285us/sample - loss: 0.0380 - acc: 0.9843
Epoch 7/10
36503/36503==============================] - 10s 283us/sample - loss: 0.0372 - acc: 0.9845
Epoch 8/10
36503/36503==============================] - 10s 285us/sample - loss: 0.0365 - acc: 0.9848
Epoch 9/10
36503/36503==============================] - 10s 283us/sample - loss: 0.0379 - acc: 0.9844
Epoch 10/10
36503/36503==============================] - 10s 283us/sample - loss: 

### Words + Context

In [ ]:
words_context_model_2 = words_context_model_builder.create_model()

In [ ]:
words_context_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
words_context_model_2.save('models/coreference_classifiers/words_context_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 27s 731us/sample - loss: 0.0725 - acc: 0.9813
Epoch 2/5
36503/36503==============================] - 26s 699us/sample - loss: 0.0436 - acc: 0.9829
Epoch 3/5
36503/36503==============================] - 26s 704us/sample - loss: 0.0386 - acc: 0.9833
Epoch 4/5
36503/36503==============================] - 25s 698us/sample - loss: 0.0376 - acc: 0.9843
Epoch 5/5
36503/36503==============================] - 26s 702us/sample - loss: 0.0345 - acc: 0.9862


In [ ]:
words_context_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22], label_gilang, epochs=5)
words_context_model_2.save('models/coreference_classifiers/words_context_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 25s 696us/sample - loss: 0.0317 - acc: 0.9869
Epoch 2/5
36503/36503==============================] - 26s 700us/sample - loss: 0.0292 - acc: 0.9879
Epoch 3/5
36503/36503==============================] - 26s 700us/sample - loss: 0.0291 - acc: 0.9879
Epoch 4/5
36503/36503==============================] - 26s 700us/sample - loss: 0.0268 - acc: 0.9895
Epoch 5/5
36503/36503==============================] - 26s 699us/sample - loss: 0.0243 - acc: 0.9895


In [ ]:
words_context_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22], label_gilang, epochs=10)
words_context_model_2.save('models/coreference_classifiers/words_context_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 26s 699us/sample - loss: 0.0233 - acc: 0.9895
Epoch 2/10
36503/36503==============================] - 25s 698us/sample - loss: 0.0235 - acc: 0.9902
Epoch 3/10
36503/36503==============================] - 26s 700us/sample - loss: 0.0235 - acc: 0.9896
Epoch 4/10
36503/36503==============================] - 26s 702us/sample - loss: 0.0221 - acc: 0.9906
Epoch 5/10
36503/36503==============================] - 26s 699us/sample - loss: 0.0202 - acc: 0.9908
Epoch 6/10
36503/36503==============================] - 25s 697us/sample - loss: 0.0221 - acc: 0.9910
Epoch 7/10
36503/36503==============================] - 25s 698us/sample - loss: 0.0198 - acc: 0.9917
Epoch 8/10
36503/36503==============================] - 26s 701us/sample - loss: 0.0192 - acc: 0.9912
Epoch 9/10
36503/36503==============================] - 26s 700us/sample - loss: 0.0195 - acc: 0.9916
Epoch 10/10
36503/36503==============================] - 26s 700us/sample - loss: 

### Words + Syntactic

In [ ]:
words_syntactic_model_2 = words_syntactic_model_builder.create_model()

In [ ]:
words_syntactic_model_2.fit([text_12, text_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_syntactic_model_2.save('models/coreference_classifiers/words_syntactic_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 18s 496us/sample - loss: 0.0617 - acc: 0.9821
Epoch 2/5
36503/36503==============================] - 17s 474us/sample - loss: 0.0408 - acc: 0.9830
Epoch 3/5
36503/36503==============================] - 17s 474us/sample - loss: 0.0376 - acc: 0.9828
Epoch 4/5
36503/36503==============================] - 17s 471us/sample - loss: 0.0352 - acc: 0.9831
Epoch 5/5
36503/36503==============================] - 17s 472us/sample - loss: 0.0332 - acc: 0.9831


In [ ]:
words_syntactic_model_2.fit([text_12, text_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_syntactic_model_2.save('models/coreference_classifiers/words_syntactic_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 17s 471us/sample - loss: 0.0299 - acc: 0.9842
Epoch 2/5
36503/36503==============================] - 17s 472us/sample - loss: 0.0282 - acc: 0.9846
Epoch 3/5
36503/36503==============================] - 17s 472us/sample - loss: 0.0291 - acc: 0.9856
Epoch 4/5
36503/36503==============================] - 17s 474us/sample - loss: 0.0283 - acc: 0.9857
Epoch 5/5
36503/36503==============================] - 17s 473us/sample - loss: 0.0290 - acc: 0.9849


In [ ]:
words_syntactic_model_2.fit([text_12, text_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
words_syntactic_model_2.save('models/coreference_classifiers/words_syntactic_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 17s 472us/sample - loss: 0.0268 - acc: 0.9857
Epoch 2/10
36503/36503==============================] - 17s 474us/sample - loss: 0.0263 - acc: 0.9859
Epoch 3/10
36503/36503==============================] - 17s 472us/sample - loss: 0.0266 - acc: 0.9858
Epoch 4/10
36503/36503==============================] - 17s 474us/sample - loss: 0.0278 - acc: 0.9859
Epoch 5/10
36503/36503==============================] - 17s 470us/sample - loss: 0.0266 - acc: 0.9857
Epoch 6/10
36503/36503==============================] - 17s 472us/sample - loss: 0.0259 - acc: 0.9862
Epoch 7/10
36503/36503==============================] - 17s 472us/sample - loss: 0.0254 - acc: 0.9869
Epoch 8/10
36503/36503==============================] - 17s 472us/sample - loss: 0.0247 - acc: 0.9878
Epoch 9/10
36503/36503==============================] - 17s 472us/sample - loss: 0.0245 - acc: 0.9877
Epoch 10/10
36503/36503==============================] - 17s 472us/sample - loss: 

### Context + Syntactic

In [ ]:
context_syntactic_model_2 = context_syntactic_model_builder.create_model()

In [ ]:
context_syntactic_model_2.fit([prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
context_syntactic_model_2.save('models/coreference_classifiers/context_syntactic_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 25s 696us/sample - loss: 0.0745 - acc: 0.9814
Epoch 2/5
36503/36503==============================] - 24s 663us/sample - loss: 0.0517 - acc: 0.9827
Epoch 3/5
36503/36503==============================] - 24s 665us/sample - loss: 0.0459 - acc: 0.9829
Epoch 4/5
36503/36503==============================] - 24s 666us/sample - loss: 0.0427 - acc: 0.9829
Epoch 5/5
36503/36503==============================] - 24s 660us/sample - loss: 0.0393 - acc: 0.9832


In [ ]:
context_syntactic_model_2.fit([prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
context_syntactic_model_2.save('models/coreference_classifiers/context_syntactic_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 24s 661us/sample - loss: 0.0383 - acc: 0.9847
Epoch 2/5
36503/36503==============================] - 24s 662us/sample - loss: 0.0340 - acc: 0.9865
Epoch 3/5
36503/36503==============================] - 24s 662us/sample - loss: 0.0315 - acc: 0.9873
Epoch 4/5
36503/36503==============================] - 24s 663us/sample - loss: 0.0303 - acc: 0.9889
Epoch 5/5
36503/36503==============================] - 24s 662us/sample - loss: 0.0289 - acc: 0.9894


In [ ]:
context_syntactic_model_2.fit([prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
context_syntactic_model_2.save('models/coreference_classifiers/context_syntactic_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 24s 661us/sample - loss: 0.0263 - acc: 0.9902
Epoch 2/10
36503/36503==============================] - 24s 666us/sample - loss: 0.0258 - acc: 0.9899
Epoch 3/10
36503/36503==============================] - 24s 660us/sample - loss: 0.0259 - acc: 0.9903
Epoch 4/10
36503/36503==============================] - 24s 661us/sample - loss: 0.0234 - acc: 0.9909
Epoch 5/10
36503/36503==============================] - 24s 669us/sample - loss: 0.0217 - acc: 0.9917
Epoch 6/10
36503/36503==============================] - 24s 663us/sample - loss: 0.0201 - acc: 0.9922
Epoch 7/10
36503/36503==============================] - 24s 666us/sample - loss: 0.0209 - acc: 0.9922
Epoch 8/10
36503/36503==============================] - 24s 661us/sample - loss: 0.0201 - acc: 0.9924
Epoch 9/10
36503/36503==============================] - 24s 662us/sample - loss: 0.0209 - acc: 0.9920
Epoch 10/10
36503/36503==============================] - 24s 665us/sample - loss: 

### Words + Context + Syntactic

In [ ]:
words_context_syntactic_model_2 = words_context_syntactic_model_builder.create_model()

In [ ]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_5.model')

Epoch 1/5
36503/36503==============================] - 33s 905us/sample - loss: 0.0666 - acc: 0.9821
Epoch 2/5
36503/36503==============================] - 32s 864us/sample - loss: 0.0430 - acc: 0.9828
Epoch 3/5
36503/36503==============================] - 32s 863us/sample - loss: 0.0395 - acc: 0.9830
Epoch 4/5
36503/36503==============================] - 32s 864us/sample - loss: 0.0363 - acc: 0.9832
Epoch 5/5
36503/36503==============================] - 32s 865us/sample - loss: 0.0343 - acc: 0.9844


In [ ]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=5)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_10.model')

Epoch 1/5
36503/36503==============================] - 32s 863us/sample - loss: 0.0308 - acc: 0.9851
Epoch 2/5
36503/36503==============================] - 32s 864us/sample - loss: 0.0312 - acc: 0.9864
Epoch 3/5
36503/36503==============================] - 32s 864us/sample - loss: 0.0298 - acc: 0.9873
Epoch 4/5
36503/36503==============================] - 31s 859us/sample - loss: 0.0272 - acc: 0.9884
Epoch 5/5
36503/36503==============================] - 31s 862us/sample - loss: 0.0264 - acc: 0.9890


In [ ]:
words_context_syntactic_model_2.fit([text_12, text_22, prev_12, prev_22, next_12, next_22, syntactic_12, syntactic_22, relation_2], label_gilang, epochs=10)
words_context_syntactic_model_2.save('models/coreference_classifiers/words_context_syntactic_gilang_20.model')

Epoch 1/10
36503/36503==============================] - 31s 862us/sample - loss: 0.0238 - acc: 0.9896
Epoch 2/10
36503/36503==============================] - 31s 862us/sample - loss: 0.0234 - acc: 0.9898
Epoch 3/10
36503/36503==============================] - 32s 865us/sample - loss: 0.0231 - acc: 0.9907
Epoch 4/10
36503/36503==============================] - 32s 866us/sample - loss: 0.0236 - acc: 0.9902
Epoch 5/10
36503/36503==============================] - 32s 865us/sample - loss: 0.0217 - acc: 0.9905
Epoch 6/10
36503/36503==============================] - 32s 865us/sample - loss: 0.0243 - acc: 0.9903
Epoch 7/10
36503/36503==============================] - 32s 865us/sample - loss: 0.0209 - acc: 0.9915
Epoch 8/10
36503/36503==============================] - 32s 864us/sample - loss: 0.0211 - acc: 0.9912
Epoch 9/10
36503/36503==============================] - 32s 865us/sample - loss: 0.0200 - acc: 0.9911
Epoch 10/10
36503/36503==============================] - 32s 865us/sample - loss: 